In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import os
sys.path.append('..')
from wyscout.events import get_key_pass_events
from wyscout.match import get_match_events, get_team_matches
from wyscout.api import set_auth, set_cache_base
from dotenv import load_dotenv
load_dotenv()
set_cache_base("..")

In [ ]:
CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")
set_auth(CLIENT_ID, CLIENT_SECRET)

In [ ]:
SOUTHEND = 1687
SEASON_2022 = 188172

In [ ]:
from wyscout.team import get_team_squad
squad = get_team_squad(SOUTHEND, SEASON_2022)
squad = {p["wyId"]: p for p in squad["squad"]}

In [ ]:
def get_key_passes():
    matches = get_team_matches(SOUTHEND, SEASON_2022)
    key_pass_events = []
    for m in matches["matches"]:
        events = get_match_events(m["matchId"])
        if "events" in events:
            key_pass_events.append(get_key_pass_events(
                m, events["events"], SOUTHEND))
    return key_pass_events

In [ ]:
matches = get_key_passes()

In [ ]:
from wyscout.viz.key_passes import plot_key_passes

plot_key_passes(matches, 5, ["J. Bridge"])

In [ ]:
def get_shots():
    matches = get_team_matches(SOUTHEND, SEASON_2022)
    events_out = []
    for m in matches["matches"]:
        events = get_match_events(m["matchId"])
        if "events" in events:
            events_out.append({
                "matchId": m["matchId"],
                "date": m["date"],
                "label": m["label"],
                "events": [e for e in events["events"] if e["team"]["id"] == SOUTHEND]
            })
    return events_out

In [ ]:
match_events = get_shots()

In [31]:
event = [e for e in match_events[4]["events"] if e["type"]["primary"] == "shot"][0]

In [41]:
from wyscout.events import get_video_for_event

get_video_for_event(event)

{'matchId': 5368929,
 'hd': {'url': 'https://cdn5lb.wyscout.com/streaming/g5368929-hd.mp4?vbegin=176&vend=186&ttkid=2578245&Expires=1668643878&Signature=s9HgBbAZCHgWZokGAjEsnO3k23qhHgtFvH8Y17T~Iiwwq89Kmm854v9PLj-I8Her-2BIBEmYESTZHAhW-KGvqs7i6a7L3Ezpw8dsqBF5UDvahwgP88HwTkHfSU3WNtF~RraAwkrAAXdIA~xDCAOP1va9wkNaKl7dSM9yH55Sh1Mzu3zXEQakW60-wXn2UrzA6F-RlnOiH-YTf7OXEfcKIsMV~aaR~EBapAm7va7~rjCnu9vlppMgrvW7BzuoGQ-zFSid4j3DOG-88nTAFl1qE8ZsrXfhtwEzaesQkX~Zs3XyVn27gjd4TwSivTqho382vAuZmloDtBXcAyhBVuagog__&Key-Pair-Id=APKAIUKTLL554SMHRIGQ'},
 'offsets': {'1H': {'start': 0, 'end': 2834},
  '2H': {'start': 2838, 'end': 5722},
  'E1': {'start': 0, 'end': 0},
  'E2': {'start': 0, 'end': 0},
  'P': {'start': 0, 'end': 0}}}

In [36]:
for i, m in enumerate([m["label"] for m in match_events]):
    print(i,m)

0 Southend United - Dorking Wanderers, 2-0
1 Southend United - Notts County, 2-2
2 Barnet - Southend United, 0-3
3 York City - Southend United, 0-2
4 Maidstone United - Southend United, 0-3
5 Southend United - Scunthorpe United, 3-0
6 Dagenham & Redbridge - Southend United, 1-1
7 Southend United - Woking, 1-1
8 Southend United - Yeovil Town, 1-0
9 Wealdstone - Southend United, 0-1
10 Southend United - Wrexham, 0-0
11 Chesterfield - Southend United, 3-2
12 Southend United - Torquay United, 1-2
13 Southend United - Maidenhead United, 2-0
14 Eastleigh - Southend United, 2-1
15 Southend United - Oldham Athletic, 1-0
16 Halifax Town - Southend United, 0-0
17 Solihull Moors - Southend United, 1-1
18 Southend United - Boreham Wood, 0-1


In [ ]:
from wyscout.viz.shots import plot_shots_compare

plot_shots_compare(match_events, 11)

In [ ]:
from statistics import mean
for i, m in enumerate(match_events[:]):
    shots = [e for e in m["events"] if e["type"]["primary"] in ["shot"]]
    m["total_shots"] = len(shots)
    m["goals"] = len([s for s in shots if s["shot"]["isGoal"] == True ])
    m["average_xg"] = mean([s["shot"]["xg"] for s in shots])
    m["total_xg"] = sum([s["shot"]["xg"] for s in shots])

In [ ]:
for m in match_events:
    print(m["label"], m["total_shots"], m["goals"], m["average_xg"], m["total_xg"])

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_axes([0,0,1,1])

ax.bar(
    [m["label"] for m in match_events],
    [m["average_xg"] for m in match_events],
    color=["blue"]
)
plt.xticks(rotation=90)
plt.title("Average xG per attempt")
plt.ylabel("xG")
plt.style.use("fivethirtyeight")
plt.show()